In [1]:
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

In [2]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [3]:
weights_file = "inception_v3.h5"

In [4]:
urllib.request.urlretrieve(weights_url, weights_file)

('inception_v3.h5', <http.client.HTTPMessage at 0x7ff41323a810>)

In [5]:
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,
                                weights=None)

In [6]:
pre_trained_model.load_weights(weights_file)

In [7]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [8]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [9]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [10]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024,activation='relu')(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1,activation='sigmoid')(x)

In [11]:
model = Model(pre_trained_model.input,x)

In [12]:
model.compile(optimizer=RMSprop(lr=0.0001),
 loss='binary_crossentropy',
 metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [13]:
training_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(training_url, training_file_name)
zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [14]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

('validation-horse-or-human.zip', <http.client.HTTPMessage at 0x7ff40e946810>)

In [15]:
zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [16]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [17]:
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1.0/255.)

In [18]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(training_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

Found 1027 images belonging to 2 classes.


In [19]:
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode='binary',
                                                         target_size=(150, 150))

Found 256 images belonging to 2 classes.


In [20]:
history = model.fit_generator(
            train_generator,
            validation_data=validation_generator,
            epochs=20,
            verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/20
52/52 [==============================] - 75s 1s/step - loss: 0.1506 - acc: 0.9494 - val_loss: 0.0438 - val_acc: 0.9844
Epoch 2/20
52/52 [==============================] - 71s 1s/step - loss: 0.0261 - acc: 0.9883 - val_loss: 0.0068 - val_acc: 1.0000
Epoch 3/20
52/52 [==============================] - 70s 1s/step - loss: 0.0175 - acc: 0.9942 - val_loss: 0.0276 - val_acc: 0.9922
Epoch 4/20
52/52 [==============================] - 70s 1s/step - loss: 0.0295 - acc: 0.9922 - val_loss: 0.0051 - val_acc: 0.9961
Epoch 5/20
52/52 [==============================] - 70s 1s/step - loss: 0.0275 - acc: 0.9922 - val_loss: 0.0191 - val_acc: 0.9961
Epoch 6/20
52/52 [==============================] - 70s 1s/step - loss: 0.0120 - acc: 0.9981 - val_loss: 0.0802 - val_acc: 0.9766
Epoch 7/20
52/52 [==============================] - 71s 1s/step - loss: 0.0081 - acc: 0.9942 - val_loss: 0.0081 - val_acc: 0.9922
Epoch 8/20
52/52 [==============================] - 70s 1s/step - loss: 0.0088 - acc: 0.99

In [1]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")